In [1]:
import numpy as np
import re
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
from collections import defaultdict

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, SGDRegressor
from sklearn import metrics

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

%matplotlib inline

In [2]:
team_lookup = pickle.load(open('teams_lookup.pickle', "rb" ))
df = pickle.load(open('final_19.pickle', "rb" ))

In [22]:
def TVT_split(df, train_min = 12, val_min = 24, test_min = 28):
    
    df_train = df[(df.GP >= train_min) & (df.GP_vs >= train_min) & (df.GP <= (val_min -1))] 
    df_val = df[(df.GP >= val_min) & (df.GP <= (test_min -1))] 
    df_test = df[df.GP >= test_min]
    
    total = len(df)
    train = len(df_train)
    val = len(df_val)
    test = len(df_test)
    mature = train + val + test
    immature = total - mature
    
    
    splits = [['dataset','games','percent of total'],['total', total ,'1'],['immature', immature, immature/total],
               ['mature', mature, mature/total], ['dataset','games','percent of mature'], ['train', train, train/mature],
               ['val', val, val/mature], ['test', test, test/mature]]
    print(splits)
    return df_train, df_val, df_test

In [23]:
df_train, df_val, df_test = TVT_split(df)

[['dataset', 'games', 'percent of total'], ['total', 11206, '1'], ['immature', 4426, 0.39496698197394253], ['mature', 6780, 0.6050330180260575], ['dataset', 'games', 'percent of mature'], ['train', 4046, 0.596755162241888], ['val', 1394, 0.2056047197640118], ['test', 1340, 0.1976401179941003]]


In [59]:
cols = df.columns.tolist()
target = 'Spread'
all_predictors = ['Home', 'Away', 'Wins', 'Tm', 'Opp', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB',
                   'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'ORtg', 'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'TRB%', 'AST%',
                   'STL%', 'BLK%', 'OeFG%', 'OTOV%', 'ORB%', 'OFT/FGA', 'DeFG%', 'DTOV%', 'DRB%', 'DFT/FGA', 'P_S', 'FG_S',
                   'FGA_S', 'FG%_S', '3P_S', '3PA_S', '3P%_S', 'FT_S', 'FTA_S', 'FT%_S', 'TRB_S', 'AST_S', 'STL_S', 'BLK_S',
                   'TOV_S', 'PF_S', 'FTr_S', '3PAr_S', 'TS%_S', 'TRB%_S', 'AST%_S', 'STL%_S', 'BLK%_S',
                   'Home_vs', 'Away_vs', 'Wins_vs', 'Tm_vs', 'Opp_vs', 'FG_vs', 'FGA_vs', 'FG%_vs', '3P_vs', '3PA_vs',
                   '3P%_vs', 'FT_vs', 'FTA_vs', 'FT%_vs', 'ORB_vs', 'TRB_vs', 'AST_vs', 'STL_vs', 'BLK_vs', 'TOV_vs',
                   'PF_vs', 'ORtg_vs', 'DRtg_vs', 'Pace_vs', 'FTr_vs', '3PAr_vs', 'TS%_vs', 'TRB%_vs', 'AST%_vs',
                   'STL%_vs', 'BLK%_vs', 'OeFG%_vs', 'OTOV%_vs', 'ORB%_vs', 'OFT/FGA_vs', 'DeFG%_vs', 'DTOV%_vs',
                   'DRB%_vs', 'DFT/FGA_vs', 'P_S_vs', 'FG_S_vs', 'FGA_S_vs', 'FG%_S_vs', '3P_S_vs', '3PA_S_vs',
                   '3P%_S_vs', 'FT_S_vs', 'FTA_S_vs', 'FT%_S_vs', 'TRB_S_vs', 'AST_S_vs', 'STL_S_vs', 'BLK_S_vs',
                   'TOV_S_vs', 'PF_S_vs', 'FTr_S_vs', '3PAr_S_vs', 'TS%_S_vs', 'TRB%_S_vs', 'AST%_S_vs', 'STL%_S_vs',
                   'BLK%_S_vs']
info_cols = ['url','Team','Date','Opponent','Win_Loss','GP','GP_vs']

## Initial Model - OLS using all stats

In [71]:
scaler = StandardScaler()

In [81]:
# Create your model
model = sm.OLS(df_train[target], sm.add_constant(scaler.fit_transform(df_train[all_predictors])))

# Fit your model to your training set
fit = model.fit()

In [82]:
# Print summary statistics of the model's performance
fit.summary(xname = ['const'] + all_predictors)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Spread   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     17.72
Date:                Wed, 15 Jul 2020   Prob (F-statistic):          2.33e-278
Time:                        00:06:07   Log-Likelihood:                -15469.
No. Observations:                4046   AIC:                         3.118e+04
Df Residuals:                    3927   BIC:                         3.193e+04
Df Model:                         118                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0788      0.177     -0.446      0.655      -0.425       0.268
Home           0.8447      0.827      1.021      0.307      -0.777       2.467
Away          -0.7596      0.827     -0.918      0.359      -2.382       0.863
Wins           0.7088      0.521      1.359      0.174      -0.313       1.731
Tm           -36.2420     75.430     -0.480      0.631    -184.128     111.644
Opp           79.5928     79.664      0.999      0.318     -76.594     235.780
FG           -33.3656    109.196     -0.306      0.760    -247.452     180.721
FGA            2.2978      4.715      0.487      0.626      -6.946      11.542
FG%          -23.2144     16.061     -1.445      0.148     -54.703       8.274
3P           -18.1655     34.141     -0.532      0.595     -85.102      48.771
3PA            0.6039      5.199      0.116      0.908      -9.590      10.797
3P%           -0.5621      1.209     -0.465      0.642      -2.931       1.807
FT            -0.9448     58.681     -0.016      0.987    -115.992     114.103
FTA          -14.4778      6.991     -2.071      0.038     -28.184      -0.772
FT%            0.9098      0.790      1.151      0.250      -0.640       2.459
ORB           -3.9107      2.087     -1.873      0.061      -8.003       0.182
TRB            0.9292      1.616      0.575      0.565      -2.240       4.098
AST            2.3652      2.194      1.078      0.281      -1.936       6.667
STL          -12.7691      5.663     -2.255      0.024     -23.872      -1.666
BLK           -1.0629      1.817     -0.585      0.558      -4.624       2.499
TOV            2.1704      4.771      0.455      0.649      -7.183      11.524
PF             2.6114      0.707      3.693      0.000       1.225       3.998
ORtg          -3.1759      5.966     -0.532      0.594     -14.872       8.520
DRtg           4.9690      5.734      0.867      0.386      -6.273      16.211
Pace           0.6785      0.941      0.721      0.471      -1.167       2.524
FTr         -178.7875    163.832     -1.091      0.275    -499.991     142.416
3PAr          -0.8285      4.819     -0.172      0.864     -10.276       8.619
TS%          -17.4807     10.016     -1.745      0.081     -37.117       2.156
TRB%          -1.4404      1.613     -0.893      0.372      -4.604       1.723
AST%          -2.0511      1.513     -1.356      0.175      -5.017       0.915
STL%          14.2257      5.385      2.642      0.008       3.669      24.783
BLK%           1.4955      1.691      0.884      0.377      -1.820       4.811
OeFG%         12.4603     14.756      0.844      0.398     -16.469      41.390
OTOV%          0.3781      3.736      0.101      0.919      -6.946       7.703
ORB%           3.1239      2.369      1.319      0.187      -1.520       7.768
OFT/FGA       -8.2147      6.605     -1.244      0.214     -21.165       4.735
De

## Ridge Regression

In [98]:
model = sm.OLS(df_train[target], df_train[all_predictors])
fit_ridge = model.fit_regularized(method='elastic_net',L1_wt = 0)

In [103]:
print(fit_ridge.summary())

NotImplementedError: 